In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import gc
import time

%matplotlib inline

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns


pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 200)


In [ ]:
app_train = pd.read_csv('../input/home-credit-default-risk/application_train.csv')
app_test = pd.read_csv('../input/home-credit-default-risk/application_test.csv')

Application_train에 대한 EDA 진행


In [ ]:
app_train['REGION_RATING_CLIENT'].value_counts()

In [ ]:
app_train['REGION_RATING_CLIENT'].astype(int).plot.hist();

연속형 숫자 feature들에 대해서 REGION_RATING_CLIENT값이 각각 1, 2, 3일때의 Histogram 시각화

In [ ]:
def show_hist_by_target(df, columns):
    cond_1 = (df['REGION_RATING_CLIENT'] == 1)
    cond_2 = (df['REGION_RATING_CLIENT'] == 2)
    cond_3 = (df['REGION_RATING_CLIENT'] == 3)
    
    for column in columns:
        fig, ax = plt.subplots(figsize=(12, 4), nrows=1, ncols=2, squeeze=False)
        sns.violinplot(x='REGION_RATING_CLIENT', y=column, data=df, ax=ax[0][0])
        sns.distplot(df[cond_1][column], label='1', color='red', ax=ax[0][1])
        sns.distplot(df[cond_2][column], label='2', color='blue', ax=ax[0][1])
        sns.distplot(df[cond_3][column], label='3', color='green', ax=ax[0][1])

In [ ]:
# Feature Importances 에서 중요하게 나왔던 칼럼만 가져옴
columns = ['AMT_INCOME_TOTAL','AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_ID_PUBLISH',
           'DAYS_REGISTRATION', 'DAYS_LAST_PHONE_CHANGE', 'CNT_FAM_MEMBERS', 'REGION_RATING_CLIENT', 'EXT_SOURCE_1', 
           'EXT_SOURCE_2', 'EXT_SOURCE_3', 'AMT_REQ_CREDIT_BUREAU_HOUR', 'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_WEEK', 
           'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT', 'AMT_REQ_CREDIT_BUREAU_YEAR']

show_hist_by_target(app_train, columns)

In [ ]:
# DAYS_BIRTH의 절대값과 REGION_RATING_CLIENT변수와의 상관계수
app_train['DAYS_BIRTH']=abs(app_train['DAYS_BIRTH'])
app_train['DAYS_BIRTH'].corr(app_train['REGION_RATING_CLIENT'])

In [ ]:
plt.style.use('fivethirtyeight')

# 고객 나이에 대한 히스토그램 분포 확인
plt.hist(app_train['DAYS_BIRTH']/365, edgecolor='k',bins=25)
plt.title('Age of Client');
plt.xlabel('Age (years)');
plt.ylabel('Count');

In [ ]:
plt.figure(figsize=(10,8))

# REGION RATE가 1인 CLIENT의 나이 plot(ranking=1)
sns.kdeplot(app_train.loc[app_train['REGION_RATING_CLIENT']==1,'DAYS_BIRTH']/365,label='rainking==1')

# REGION RATE가 2인 CLIENT의 나이 plot(ranking=2)
sns.kdeplot(app_train.loc[app_train['REGION_RATING_CLIENT']==2,'DAYS_BIRTH']/365,label='rainking==2')

# REGION RATE가 3인 CLIENT의 나이 plot(ranking=3)
sns.kdeplot(app_train.loc[app_train['REGION_RATING_CLIENT']==3,'DAYS_BIRTH']/365,label='rainking==3')


plt.xlabel('Age(years)');
plt.ylabel('Density');
plt.title('Distribution of Ages');

Target을 REGION_RATING_CLIENT 유형에 따라 Count 비교

In [ ]:
object_columns = app_train.dtypes[app_train.dtypes == 'object'].index.tolist()
object_columns

In [ ]:
sns.catplot(x='TARGET', col='REGION_RATING_CLIENT', data=app_train, kind='count')

In [ ]:
# REGION RATING이 연체 했을 때 더 높음--> 이를 value_counts()로 확인

cond_1 = (app_train['REGION_RATING_CLIENT'] == 1)
cond_2 = (app_train['REGION_RATING_CLIENT'] == 2)
cond_3 = (app_train['REGION_RATING_CLIENT'] == 3)
print(app_train['TARGET'].value_counts()/app_train.shape[0])
print('\nRanking이 1일 때\n',app_train[cond_1]['TARGET'].value_counts()/app_train[cond_1].shape[0])
print('\nRanking이 2일 때\n',app_train[cond_2]['TARGET'].value_counts()/app_train[cond_1].shape[0])
print('\nRanking이 3일 때\n',app_train[cond_3]['TARGET'].value_counts()/app_train[cond_1].shape[0])

데이터 가공 전 학습과 테스트용 데이터 세트 결합

In [ ]:
apps = pd.concat([app_train, app_test])
print(apps.shape)

In [ ]:
# 데이터 레이블 인코딩, NULL값은 LightGBM 내부에서 처리하도록 특별한 변경하지 않음

object_columns = apps.dtypes[apps.dtypes=='object'].index.tolist()

for column in object_columns:
    apps[column] = pd.factorize(apps[column])[0]

학습 데이터와 테스트 데이터 다시 분리

In [ ]:
apps_train = apps[-apps['TARGET'].isnull()]
apps_test = apps[apps['TARGET'].isnull()]
apps.shape, apps_train.shape, apps_test.shape


학습 데이터를 검증 데이터로 분리하고 LGBM Classifier로 학습 수행

In [ ]:
from sklearn.model_selection import train_test_split

ftr_app = apps_train.drop(['SK_ID_CURR', 'TARGET'], axis=1)
target_app = app_train['TARGET']

train_x, valid_x, train_y, valid_y = train_test_split(ftr_app, target_app, test_size=0.3, random_state=2020)
train_x.shape, valid_x.shape

In [ ]:
from lightgbm import LGBMClassifier

clf = LGBMClassifier(
        n_jobs=-1,
        n_estimators=1000,
        learning_rate=0.02,
        num_leaves=32,
        subsample=0.8,
        max_depth=12,
        silent=-1,
        verbose=-1
        )

clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], eval_metric= 'auc', verbose= 100, 
        early_stopping_rounds= 100)

In [ ]:
from lightgbm import plot_importance

plot_importance(clf, figsize=(16, 32))

학습된 Classifier를 이용하여 테스트 데이터 예측하고 결과를 Kaggle로 Submit 수행

In [ ]:
preds = clf.predict_proba(apps_test.drop(['SK_ID_CURR', 'TARGET'], axis=1))[:, 1 ]

In [ ]:
app_test['TARGET'] = preds
app_test[['SK_ID_CURR', 'TARGET']].to_csv('apps_baseline_02.csv', index=False)